# Quora question pairs: training

## Import packages

In [1]:
%matplotlib inline
from __future__ import print_function
import numpy as np
import pandas as pd
import datetime, time, json, csv
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from keras.layers import Input, TimeDistributed, Dense, Lambda, concatenate, Dropout, BatchNormalization
from keras.layers.embeddings import Embedding
from keras.regularizers import l2
from keras.callbacks import Callback, ModelCheckpoint
from keras import backend as K
from sklearn.model_selection import train_test_split

/home/ubuntu/anaconda3/envs/Keras_GPU/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ubuntu/anaconda3/envs/Keras_GPU/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ubuntu/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))
/home/ubuntu/anaconda3/envs/Keras_GPU/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Initialize global variables

In [2]:
Q1_TRAINING_DATA_FILE = 'q1_train.npy'
Q2_TRAINING_DATA_FILE = 'q2_train.npy'
LABEL_TRAINING_DATA_FILE = 'label_train.npy'
WORD_EMBEDDING_MATRIX_FILE = 'word_embedding_matrix.npy'
NB_WORDS_DATA_FILE = 'nb_words.json'
MODEL_WEIGHTS_FILE = 'question_pairs_weights.h5'
MAX_SEQUENCE_LENGTH = 25
EMBEDDING_DIM = 300
VALIDATION_SPLIT = 0.1
TEST_SPLIT = 0.1
RNG_SEED = 13371447
NB_EPOCHS = 25
DROPOUT = 0.1
BATCH_SIZE = 32

## Load the dataset, embedding matrix and word count

In [3]:
q1_data = np.load(open(Q1_TRAINING_DATA_FILE, 'rb'))
q2_data = np.load(open(Q2_TRAINING_DATA_FILE, 'rb'))
labels = np.load(open(LABEL_TRAINING_DATA_FILE, 'rb'))
word_embedding_matrix = np.load(open(WORD_EMBEDDING_MATRIX_FILE, 'rb'))
with open(NB_WORDS_DATA_FILE, 'r') as f:
    nb_words = json.load(f)['nb_words']

## Partition the dataset into train and test sets

In [4]:
X = np.stack((q1_data, q2_data), axis=1)
y = labels
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=TEST_SPLIT, random_state=RNG_SEED)
Q1_train = X_train[:,0]
Q2_train = X_train[:,1]
Q1_test = X_test[:,0]
Q2_test = X_test[:,1]

## Define the model

In [5]:
question1 = Input(shape=(MAX_SEQUENCE_LENGTH,))
question2 = Input(shape=(MAX_SEQUENCE_LENGTH,))

q1 = Embedding(nb_words + 1, 
                 EMBEDDING_DIM, 
                 weights=[word_embedding_matrix], 
                 input_length=MAX_SEQUENCE_LENGTH, 
                 trainable=False)(question1)
q1 = TimeDistributed(Dense(EMBEDDING_DIM, activation='relu'))(q1)
q1 = Lambda(lambda x: K.max(x, axis=1), output_shape=(EMBEDDING_DIM, ))(q1)

q2 = Embedding(nb_words + 1, 
                 EMBEDDING_DIM, 
                 weights=[word_embedding_matrix], 
                 input_length=MAX_SEQUENCE_LENGTH, 
                 trainable=False)(question2)
q2 = TimeDistributed(Dense(EMBEDDING_DIM, activation='relu'))(q2)
q2 = Lambda(lambda x: K.max(x, axis=1), output_shape=(EMBEDDING_DIM, ))(q2)

merged = concatenate([q1,q2])
merged = Dense(200, activation='relu')(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)
merged = Dense(200, activation='relu')(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)
merged = Dense(200, activation='relu')(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)
merged = Dense(200, activation='relu')(merged)
merged = Dropout(DROPOUT)(merged)
merged = BatchNormalization()(merged)

is_duplicate = Dense(1, activation='sigmoid')(merged)

model = Model(inputs=[question1,question2], outputs=is_duplicate)
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

## Model Summary

In [6]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 25)           0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 25)           0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 25, 300)      28679100    input_1[0][0]                    
__________________________________________________________________________________________________
embedding_2 (Embedding)         (None, 25, 300)      28679100    input_2[0][0]                    
__________________________________________________________________________________________________
time_distr

## Train the model, checkpointing weights with best validation accuracy

In [7]:
print("Starting training at", datetime.datetime.now())
t0 = time.time()
callbacks = [ModelCheckpoint(MODEL_WEIGHTS_FILE, monitor='val_acc', save_best_only=True)]
history = model.fit([Q1_train, Q2_train],
                    y_train,
                    epochs=NB_EPOCHS,
                    validation_split=VALIDATION_SPLIT,
                    verbose=2,
                    batch_size=BATCH_SIZE,
                    callbacks=callbacks)
t1 = time.time()
print("Training ended at", datetime.datetime.now())
print("Minutes elapsed: %f" % ((t1 - t0) / 60.))

Starting training at 2018-04-09 06:59:44.898728
Train on 327474 samples, validate on 36387 samples
Epoch 1/25
 - 140s - loss: 0.5364 - acc: 0.7284 - val_loss: 0.5116 - val_acc: 0.7485
Epoch 2/25
 - 138s - loss: 0.4831 - acc: 0.7641 - val_loss: 0.4745 - val_acc: 0.7715
Epoch 3/25
 - 138s - loss: 0.4542 - acc: 0.7814 - val_loss: 0.4456 - val_acc: 0.7828
Epoch 4/25
 - 138s - loss: 0.4310 - acc: 0.7950 - val_loss: 0.4408 - val_acc: 0.7885
Epoch 5/25
 - 138s - loss: 0.4141 - acc: 0.8061 - val_loss: 0.4250 - val_acc: 0.7967
Epoch 6/25
 - 138s - loss: 0.3978 - acc: 0.8160 - val_loss: 0.6183 - val_acc: 0.6904
Epoch 7/25
 - 138s - loss: 0.3833 - acc: 0.8245 - val_loss: 0.4155 - val_acc: 0.7993
Epoch 8/25
 - 138s - loss: 0.3713 - acc: 0.8311 - val_loss: 0.4159 - val_acc: 0.7995
Epoch 9/25
 - 138s - loss: 0.3597 - acc: 0.8381 - val_loss: 0.4336 - val_acc: 0.7929
Epoch 10/25
 - 138s - loss: 0.3486 - acc: 0.8446 - val_loss: 0.4037 - val_acc: 0.8089
Epoch 11/25
 - 138s - loss: 0.3392 - acc: 0.8497 -

## Print best validation accuracy and epoch

In [8]:
max_val_acc, idx = max((val, idx) for (idx, val) in enumerate(history.history['val_acc']))
print('Maximum accuracy at epoch', '{:d}'.format(idx+1), '=', '{:.4f}'.format(max_val_acc))

Maximum accuracy at epoch 25 = 0.8158


## Evaluate the model with best validation accuracy on the test partition

In [9]:
model.load_weights(MODEL_WEIGHTS_FILE)
loss, accuracy = model.evaluate([Q1_test, Q2_test], y_test, verbose=0)
print('loss = {0:.4f}, accuracy = {1:.4f}'.format(loss, accuracy))

loss = 0.4108, accuracy = 0.8161


# Generating Predication values for test.csv

In [14]:
#Please Download and extract the test.csv file where you have placed these notebooks otherwise the following code won't execute
TESTING_FILE='test.csv'
OUTPUT_FILE= "OUTPUT_FILE.csv"
MAX_NB_WORDS = 200000
MAX_SEQUENCE_LENGTH = 25

print("Processing", TESTING_FILE)

testid = []
question1 = []
question2 = []

with open(TESTING_FILE, encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile, delimiter=',')
    for row in reader:
        testid.append(row['test_id'])
        question1.append(row['question1'])
        question2.append(row['question2'])
        
questions = question1 + question2
tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(questions)
question1_word_sequences = tokenizer.texts_to_sequences(question1)
question2_word_sequences = tokenizer.texts_to_sequences(question2)
word_index = tokenizer.word_index

print("Words in index: %d" % len(word_index))

q1_data_test = pad_sequences(question1_word_sequences, maxlen=MAX_SEQUENCE_LENGTH)
q2_data_test = pad_sequences(question2_word_sequences, maxlen=MAX_SEQUENCE_LENGTH)
test_id = np.array(testid, dtype=int)
print('Shape of id tensor:', test_id.shape)
print('Shape of question1 test data tensor:', q1_data_test.shape)
print('Shape of question2 test data tensor:', q2_data_test.shape)


model.load_weights(MODEL_WEIGHTS_FILE)
solution=model.predict([q1_data_test, q2_data_test],batch_size=BATCH_SIZE,verbose=2)

with open(OUTPUT_FILE, "w+") as csv_file:   
    writer = csv.DictWriter(csv_file, fieldnames = ["test_id", "probability"])
    writer.writeheader()
    writer = csv.writer(csv_file, delimiter=',')
    for value in range(len(test_id)):
        output=[]
        output.append(test_id[value])
        output.append(solution[value][0])
        writer.writerow(output)      

Processing test.csv
Words in index: 101312
Shape of id tensor: (2345796,)
Shape of question1 test data tensor: (2345796, 25)
Shape of question2 test data tensor: (2345796, 25)
